##### Lab 5
Apply, Groupby operations

### Learning objectives 
Upon successful completion of this lab, you will have demonstrated the abilities to: 

- Applying functions across columns or rows of data
- Use Groupby operations to aggregate, transform, and filter data
- Use built-in and custom user functions to perform groupby operations


In [115]:
import pandas as pd
housing = pd.read_csv(r'C:\Users\Rinchen\Desktop\AIG\AIG 150\Week 05\housing.csv')
housing.head()

,Neighborhood,Building.Classification,Total.Units,Year.Built,Gross.SqFt,Estimated.Gross.Income,Gross.Income.per.SqFt,Estimated.Expense,Expense.per.SqFt,Net.Operating.Income,Full.Market.Value,Market.Value.per.SqFt,Boro
0,FINANCIAL,R9-CONDOMINIUM,42,1920.0,36500,1332615,36.51,342005,9.37,990610,7300000,200.00,Manhattan
1,FINANCIAL,R4-CONDOMINIUM,78,1985.0,126420,6633257,52.47,1762295,13.94,4870962,30690000,242.76,Manhattan
2,FINANCIAL,RR-CONDOMINIUM,500,NaN,554174,17310000,31.24,3543000,6.39,13767000,90970000,164.15,Manhattan
3,FINANCIAL,R4-CONDOMINIUM,282,1930.0,249076,11776313,47.28,2784670,11.18,8991643,67556006,271.23,Manhattan
4,TRIBECA,R4-CONDOMINIUM,239,1985.0,219495,10004582,45.58,2783197,12.68,7221385,54320996,247.48,Manhattan


#### 1. standarization 
- Transform the variable Gross.SqFt to its Zscore and 
- save it into a new column stand_Gross.SqFt. 
- print the first  obs of stand_Gross.SqFt column. 

In [117]:
from scipy.stats import zscore

housing['stand_Gross.SqFt'] = zscore(housing['Gross.SqFt'])
housing['stand_Gross.SqFt'].head()

0   -0.321278
1    0.303182
2    3.273773
3    1.154982
4    0.949553
Name: stand_Gross.SqFt, dtype: float64

#### 2. define a function 
- define a function outlierDetectionZ that returns the string literal 'low' if the z-score of the input parameter is less than -3 and return the string literal 'high' if the z-score value is greater than 3

In [118]:
def outlierDetectionZ(column):
    z_scores = zscore(column)
    result = []
    for z_score in z_scores:
        if z_score < -3:
            result.append('low')
        elif z_score > 3:
            result.append('high')
        else:
            result.append('normal')
    return result

housing['ZScore'] = outlierDetectionZ(housing['Gross.SqFt'])

selected_columns = ['Gross.SqFt', 'stand_Gross.SqFt' ,'ZScore']
result_df = housing[selected_columns]
result_df.head()

,Gross.SqFt,stand_Gross.SqFt,ZScore
0,36500,-0.321278,normal
1,126420,0.303182,normal
2,554174,3.273773,high
3,249076,1.154982,normal
4,219495,0.949553,normal


#### 3. Vectorize the function
- vectorize outlierDetectionZ function and name the vectorized function as outlierDetectionZ_vec
- call the vectorized function with housing['stand_Gross.SqFt'] as argument. Save the return value in a variable lowhigh. 
- loop through the list lowhigh, and print the obs labels with either low or high and the corresponding row from the dataframe.  

In [119]:
def outlierDetectionZ(value):
    if value < -3:
        return 'low'
    elif value > 3:
        return 'high'
    else:
        return 'normal'

outlierDetectionZ_vec = np.vectorize(outlierDetectionZ)
lowhigh = outlierDetectionZ_vec(housing['stand_Gross.SqFt'])
for idx, label in enumerate(lowhigh):
    if label == 'low' or label == 'high':
        print(label,'\n',housing.iloc[idx], '\n')

high 
 Neighborhood                    FINANCIAL
Building.Classification    RR-CONDOMINIUM
Total.Units                           500
Year.Built                            NaN
Gross.SqFt                         554174
Estimated.Gross.Income           17310000
Gross.Income.per.SqFt               31.24
Estimated.Expense                 3543000
Expense.per.SqFt                     6.39
Net.Operating.Income             13767000
Full.Market.Value                90970000
Market.Value.per.SqFt              164.15
Boro                            Manhattan
stand_Gross.SqFt                 3.273773
ZScore                               high
Name: 2, dtype: object 

high 
 Neighborhood                      TRIBECA
Building.Classification    R4-CONDOMINIUM
Total.Units                           546
Year.Built                         1987.0
Gross.SqFt                         586224
Estimated.Gross.Income           24375194
Gross.Income.per.SqFt               41.58
Estimated.Expense                 711

#### 4. Use apply 
- Instead of vectorizing the function, use apply on housing['stand_Gross.SqFt'] and pass outlierDetectionZ as argument to get obs with labels high/low. 
- show the obs that have labels low or high (as per the previous question)

In [120]:
lowhigh = housing['stand_Gross.SqFt'].apply(outlierDetectionZ)
for label, obs in zip(lowhigh, housing.iterrows()):
    if label == 'low' or label == 'high':
        print(label, obs)

high (2, Neighborhood                    FINANCIAL
Building.Classification    RR-CONDOMINIUM
Total.Units                           500
Year.Built                            NaN
Gross.SqFt                         554174
Estimated.Gross.Income           17310000
Gross.Income.per.SqFt               31.24
Estimated.Expense                 3543000
Expense.per.SqFt                     6.39
Net.Operating.Income             13767000
Full.Market.Value                90970000
Market.Value.per.SqFt              164.15
Boro                            Manhattan
stand_Gross.SqFt                 3.273773
ZScore                               high
Name: 2, dtype: object)
high (11, Neighborhood                      TRIBECA
Building.Classification    R4-CONDOMINIUM
Total.Units                           546
Year.Built                         1987.0
Gross.SqFt                         586224
Estimated.Gross.Income           24375194
Gross.Income.per.SqFt               41.58
Estimated.Expense                

#### 5.  Use apply and lambda function 
- on the stand_Gross.SqFt column, call apply with lambda function. 
- the lambda function returns 'outlier' when the z-score is lower than -3 or larger than 3, and none otherwise.

In [132]:
housing['outlier'] = housing['stand_Gross.SqFt'].apply(lambda x: 'outlier' if (zscore([x])[0] < -3 or zscore([x])[0] > 3) else None)
selected_columns = ['Gross.SqFt','stand_Gross.SqFt', 'ZScore', 'outlier']
housing[selected_columns]

,Gross.SqFt,stand_Gross.SqFt,ZScore,outlier
0,36500,-0.321278,normal,None
1,126420,0.303182,normal,None
2,554174,3.273773,high,None
3,249076,1.154982,normal,None
4,219495,0.949553,normal,None
...,...,...,...,...
2621,62391,-0.141475,normal,None
2622,90618,0.054551,normal,None
2623,78903,-0.026805,normal,None
2624,166712,0.582995,normal,None


#### 6. Use group by and aggregate functions. 
    6.1. Show the average full market value per boro.

    6.2. Show the average full market value per boro and Neighborhood

    6.3. show the average, min and max full market value per boro and Neighborhood. Use agg function and rename the columns as min Market val, max market val and avg market val and also flatten the dataframe.


In [134]:
housing.dtypes

Neighborhood                object
Building.Classification     object
Total.Units                  int64
Year.Built                 float64
Gross.SqFt                   int64
Estimated.Gross.Income       int64
Gross.Income.per.SqFt      float64
Estimated.Expense            int64
Expense.per.SqFt           float64
Net.Operating.Income         int64
Full.Market.Value            int64
Market.Value.per.SqFt      float64
Boro                        object
stand_Gross.SqFt           float64
ZScore                      object
outlier                     object
dtype: object

In [142]:
average_market_value_per_boro = housing.groupby('Boro')['Full.Market.Value'].mean()
average_market_value_per_boro

Boro
Bronx            6.330082e+06
Brooklyn         3.028974e+06
Manhattan        2.143214e+07
Queens           4.167439e+06
Staten Island    3.313400e+06
Name: Full.Market.Value, dtype: float64

In [143]:
average_market_value_per_boro_neighborhood = housing.groupby(['Boro', 'Neighborhood'])['Full.Market.Value'].mean()
average_market_value_per_boro_neighborhood

Boro           Neighborhood        
Bronx          BEDFORD PARK/NORWOOD     1816498.0
               BELMONT                  2752000.0
               BRONX-UNKNOWN           74391091.5
               BRONXDALE                3125671.0
               CITY ISLAND               678001.0
                                          ...    
Staten Island  ROSEBANK                 3354003.0
               SILVER LAKE              2077903.0
               SUNNYSIDE                2340743.0
               TOMPKINSVILLE            5697922.0
               WEST NEW BRIGHTON        1148655.0
Name: Full.Market.Value, Length: 152, dtype: float64

In [145]:
agg_functions = {
    'Full.Market.Value': ['min', 'max', 'mean']
}
result = housing.groupby(['Boro', 'Neighborhood']).agg(agg_functions).reset_index()
result.columns = ['Boro', 'Neighborhood', 'min Market val', 'max Market val', 'avg Market val']
print(result)

              Boro          Neighborhood  min Market val  max Market val  \
0            Bronx  BEDFORD PARK/NORWOOD          651996         2981000   
1            Bronx               BELMONT         2752000         2752000   
2            Bronx         BRONX-UNKNOWN        56470051        92312132   
3            Bronx             BRONXDALE         2827000         3424007   
4            Bronx           CITY ISLAND          678001          678001   
..             ...                   ...             ...             ...   
147  Staten Island              ROSEBANK         3354003         3354003   
148  Staten Island           SILVER LAKE         1845800         2310006   
149  Staten Island             SUNNYSIDE         2340743         2340743   
150  Staten Island         TOMPKINSVILLE         5697922         5697922   
151  Staten Island     WEST NEW BRIGHTON         1148655         1148655   

     avg Market val  
0         1816498.0  
1         2752000.0  
2        74391091.5  

#### 7. grouped zscore versus nongrouped zscore. 
    7.1. Calculate a grouped zscore for full market value per Boro.  
    7.2. Also calculate a non-grouped zscore for full market value. 
    7.3. Show the first 5 obs of each series. 

#### 7.1

In [162]:
housing['zscore_grouped'] = housing.groupby('Boro')['Full.Market.Value'].transform(lambda x: zscore(x, ddof=1))

#### 7.2

In [163]:
housing['zscore'] = zscore(housing['Full.Market.Value'], ddof=1)

#### 7.3

In [166]:
print("\nFirst 5 observations of non-grouped zscore:")
print(housing['zscore'].head())

print("\nFirst 5 observations of grouped zscore:")
housing['zscore_grouped'].head()


First 5 observations of non-grouped zscore:
0   -0.239113
1    0.745925
2    3.284534
3    2.298486
4    1.741112
Name: zscore, dtype: float64

First 5 observations of grouped zscore:


0   -0.475363
1    0.311406
2    2.339044
3    1.551468
4    1.106282
Name: zscore_grouped, dtype: float64

#### 8. Filter rows based on conditions on groups
    8.1. show the frequency count (number of houses) per Boro. (use value_counts() method) 
    8.2. filter out the houses that belong to a Boro with a frequency count less than 100. name the new dataframe as housing_filtered 
    8.3. show the frequency count (number of houses) per Boro in the filtered_housing dataframe. 


#### 8.1

In [167]:
# Calculate frequency count per Boro
frequency_per_boro = housing['Boro'].value_counts()
print(frequency_per_boro)

Boro
Manhattan        1380
Brooklyn          717
Queens            434
Bronx              69
Staten Island      26
Name: count, dtype: int64


#### 8.2

In [159]:
housing_filtered = housing[housing['Boro'].map(frequency_per_boro) >= 100]
housing_filtered.head()

,Neighborhood,Building.Classification,Total.Units,Year.Built,Gross.SqFt,Estimated.Gross.Income,Gross.Income.per.SqFt,Estimated.Expense,Expense.per.SqFt,Net.Operating.Income,Full.Market.Value,Market.Value.per.SqFt,Boro,stand_Gross.SqFt,ZScore,outlier,zscore_grouped,zscore
0,FINANCIAL,R9-CONDOMINIUM,42,1920.0,36500,1332615,36.51,342005,9.37,990610,7300000,200.00,Manhattan,-0.321278,normal,None,-0.475363,-0.239113
1,FINANCIAL,R4-CONDOMINIUM,78,1985.0,126420,6633257,52.47,1762295,13.94,4870962,30690000,242.76,Manhattan,0.303182,normal,None,0.311406,0.745925
2,FINANCIAL,RR-CONDOMINIUM,500,NaN,554174,17310000,31.24,3543000,6.39,13767000,90970000,164.15,Manhattan,3.273773,high,None,2.339044,3.284534
3,FINANCIAL,R4-CONDOMINIUM,282,1930.0,249076,11776313,47.28,2784670,11.18,8991643,67556006,271.23,Manhattan,1.154982,normal,None,1.551468,2.298486
4,TRIBECA,R4-CONDOMINIUM,239,1985.0,219495,10004582,45.58,2783197,12.68,7221385,54320996,247.48,Manhattan,0.949553,normal,None,1.106282,1.741112


#### 8.3

In [161]:
frequency_per_boro_filtered = housing_filtered['Boro'].value_counts()
print(frequency_per_boro_filtered)

Boro
Manhattan    1380
Brooklyn      717
Queens        434
Name: count, dtype: int64


#### 9. Calculate the mean of Gross.SqFt and Estimated.Gross.Income by  Boro and Neighborhood
    - define a variable groups that holds grouped dataframe by Boro and Neighborhood. include only the Boro, Neighborhood, Gross.SqFt and Estimated.Gross.Income columns from the original dataset. 
    - show the mean of Gross.SqFt and Estimated.Gross.Income per group.   

In [178]:
columns_to_include = ['Boro', 'Neighborhood', 'Gross.SqFt', 'Estimated.Gross.Income']

housing_filtered = housing[columns_to_include]

groups = housing_filtered.groupby(['Boro', 'Neighborhood'])

mean_per_group = groups.mean()
print(mean_per_group)

                                      Gross.SqFt  Estimated.Gross.Income
Boro          Neighborhood                                              
Bronx         BEDFORD PARK/NORWOOD  4.827900e+04                562398.5
              BELMONT               4.875000e+04                812175.0
              BRONX-UNKNOWN         2.295597e+06              26939916.0
              BRONXDALE             1.096907e+05               1016980.0
              CITY ISLAND           1.660500e+04                218190.0
...                                          ...                     ...
Staten Island ROSEBANK              6.239100e+04                831672.0
              SILVER LAKE           5.802600e+04                739582.0
              SUNNYSIDE             6.539200e+04                857943.0
              TOMPKINSVILLE         1.590960e+05               1942562.0
              WEST NEW BRIGHTON     2.894300e+04                406939.0

[152 rows x 2 columns]


#### 10. Iterate through the groups and print the gross square feet for Bronx boro in BEDFORD PARK/NORWOOD. 
    The output should be as follows: 
        the gross square Feet for Bronx in BEDFORD PARK/NORWOOD is: 79920

In [171]:
for (boro, neighborhood), group_df in groups:
    if boro == 'Bronx' and neighborhood == 'BEDFORD PARK/NORWOOD':
        print(f"The gross square Feet for Bronx in BEDFORD PARK/NORWOOD is: {group_df['Gross.SqFt'].mean()}")

The gross square Feet for Bronx in BEDFORD PARK/NORWOOD is: 48279.0
